In [54]:
import numpy as np
from moviepy.editor import VideoFileClip, concatenate
from moviepy.editor import *

import os

import matplotlib.pyplot as plt

In [55]:
input_file = 'video.mp4'

In [56]:
clip = VideoFileClip(input_file)#.subclip(0,1000)

In [57]:
fps = 44000
a = 0.3
b = 10
out_name = 'cut_not' + '.mp4'

In [58]:
y = clip.audio.subclip(10,20).to_soundarray(fps=10000)[:,0] # only get one of the soundvawes

In [59]:
y = clip.audio.to_soundarray(fps=fps)[:,0] # only get one of the soundvawes
y = np.abs(y)

In [60]:
def get_non_silent_parts(a, b, silence=0.01):
    a_frames = int(a * fps)

    # whether frames from i*a_frames:(i+1)*a_frames are sounded
    # which is equal to i-seconds:i+a-seconds
    sounds = []

    current = 0

    for i in range(0,len(y),a_frames): # len(y)
        interval = y[i : i + a_frames]

        split_intervals = np.array(np.split(interval,b))

        means = split_intervals.mean(axis=1)

        silent = means < silence


        new_silent = np.all(silent)

        if not current:
            new_silent == 0

        sounds.append(new_silent)

        current = new_silent

        """ plt.plot(interval)
        plt.plot([(x+1/2)*a_frames/b for x in range(b)], means) """

    return np.invert(sounds)

In [61]:
def getSecondsTuples():
    sounds_array = get_non_silent_parts(a,b)
    areas = []

    current = False
    current_start = 0

    for i in range(len(sounds_array)):

        if sounds_array[i]:
            if not current:
                current_start = i
                current = True
        else:
            if current:
                areas.append((current_start*a,(i)*a))
            current = 0

    if current:
        areas.append((current_start*a,(len(sounds_array))*a))

    return areas

In [62]:
def ffmpeg_text():
    tups_array = getSecondsTuples()
    text = f"select='{'+'.join([f'between(t,{tup[0]},{tup[1]})' for tup in tups_array])}'"
    return text

In [66]:
def run_ffmpeg_script():
    text = ffmpeg_text()
    os.system(
        f"""ffmpeg -i {input_file} \
       -vf "{text},
            setpts=N/FRAME_RATE/TB" \
       -af "a{text},
            asetpts=N/SR/TB" fast -y {out_name}"""
    )
run_ffmpeg_script()

In [64]:
def makeNewVideo():
    sounds = get_non_silent_parts(a,b)
    timestamps = np.arange(0,clip.duration,a)
    new_videos = np.array([clip.subclip(i,i+a) for i,x in zip(timestamps,sounds) if x])
    return concatenate(new_videos)

In [65]:
#makeNewVideo().write_videofile(out_name, threads = 4,logger = None)